# Landing Zone Detection with RF-DETR
## CS4824 Final Project - Google Colab Training Notebook

This notebook trains an RF-DETR model to detect landing zones from Blender-rendered frames.

### Instructions:
1. Upload `coco-dataset.zip` to your Google Drive (root folder or a specific folder)
2. Run this notebook with GPU enabled: Runtime → Change runtime type → GPU (T4)
3. Mount Google Drive when prompted
4. The notebook will unzip the dataset and train the model

## 1. Setup and Installation

First, check GPU availability and install dependencies.

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Mount Google Drive & Load Dataset

Mount your Google Drive and unzip the dataset.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to your zip file in Google Drive
# Change this path if you put the zip file in a different folder
zip_path = '/content/drive/MyDrive/coco-dataset.zip'

# Alternative: If you put it in a subfolder, use something like:
# zip_path = '/content/drive/MyDrive/CS4824/coco-dataset.zip'

import os
if os.path.exists(zip_path):
    print(f"✓ Found dataset at: {zip_path}")
else:
    print(f"✗ Dataset not found at: {zip_path}")
    print("Please upload coco-dataset.zip to your Google Drive and update the path above.")

## 3. Unzip Dataset & Install RF-DETR

Extract the dataset and install the RF-DETR library.

In [ ]:
# Unzip the dataset
import zipfile

zip_path = '/content/drive/MyDrive/coco-dataset.zip'
extract_path = '/content'

print("Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✓ Dataset extracted!")

# Verify extraction
import os
dataset_path = '/content/coco-dataset'
if os.path.exists(dataset_path):
    print(f"✓ Dataset directory exists: {dataset_path}")
    for split in ['train', 'valid', 'test']:
        split_path = os.path.join(dataset_path, split)
        if os.path.exists(split_path):
            num_files = len(os.listdir(split_path))
            print(f"  {split}: {num_files} files")
else:
    print("✗ Dataset extraction failed")

## 4. Install RF-DETR

Clone and install the RF-DETR library from Roboflow.

In [ ]:
# Clone and install RF-DETR
!git clone https://github.com/roboflow/rf-detr.git
%cd rf-detr
!pip install -e .
%cd /content

print("\n✓ RF-DETR installed successfully!")

## 5. Verify Dataset Structure

Check that the COCO dataset is properly structured.

In [ ]:
import os
import json

# Check dataset structure
dataset_path = "/content/coco-dataset"

print("Dataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

# Check annotation files
print("\n" + "="*50)
for split in ['train', 'valid', 'test']:
    anno_file = f"{dataset_path}/{split}/_annotations.coco.json"
    if os.path.exists(anno_file):
        with open(anno_file, 'r') as f:
            data = json.load(f)
        print(f"\n{split.upper()} split:")
        print(f"  Images: {len(data['images'])}")
        print(f"  Annotations: {len(data['annotations'])}")
        print(f"  Categories: {data['categories']}")

## 6. Visualize Sample Images

Display some training images with their bounding boxes.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

# Load training annotations
with open('/content/coco-dataset/train/_annotations.coco.json', 'r') as f:
    train_data = json.load(f)

# Create image_id to annotations mapping
img_to_annos = {}
for anno in train_data['annotations']:
    img_id = anno['image_id']
    if img_id not in img_to_annos:
        img_to_annos[img_id] = []
    img_to_annos[img_id].append(anno)

# Display first 6 images with annotations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_info in enumerate(train_data['images'][:6]):
    img_path = f"/content/coco-dataset/train/{img_info['file_name']}"
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ax = axes[idx]
    ax.imshow(img)
    
    # Draw bounding boxes
    if img_info['id'] in img_to_annos:
        for anno in img_to_annos[img_info['id']]:
            x, y, w, h = anno['bbox']
            rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
    
    ax.set_title(f"{img_info['file_name']}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## 7. Train the RF-DETR Model

Train the model using the RF-DETR library.

In [ ]:
import os
import torch
from pathlib import Path
from rfdetr import RFDETRNano
from rfdetr.config import TrainConfig

def train_landing_zone_model():
    """Train RF-DETR model for landing zone detection"""
    
    dataset_path = "/content/coco-dataset"
    
    # Initialize model
    print("Initializing RF-DETR Nano model...")
    model = RFDETRNano(
        class_names=['Landing Zone'],
    )
    
    # Configure training
    config = TrainConfig(
        data_root=dataset_path,
        output_dir="/content/output/landing_zone_detection",
        batch_size=8,  # Good batch size for T4 GPU
        epochs=100,
        learning_rate=0.0001,
        num_workers=2,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        image_size=640,
        save_interval=10,
        eval_interval=5,
        warmup_epochs=5
    )
    
    print(f"\nTraining Configuration:")
    print(f"  Device: {config.device}")
    print(f"  Batch size: {config.batch_size}")
    print(f"  Epochs: {config.epochs}")
    print(f"  Learning rate: {config.learning_rate}")
    print(f"  Image size: {config.image_size}")
    print(f"  Data root: {config.data_root}")
    print(f"  Output dir: {config.output_dir}\n")
    
    # Train model
    print("Starting training...")
    model.train(config)
    
    print("\n✓ Training completed!")
    print(f"Model saved to: {config.output_dir}")
    
    return model, config

# Run training
model, config = train_landing_zone_model()

## 8. Visualize Training Results

Check the output directory for trained model checkpoints.

In [ ]:
# Check for training outputs
import os

output_dir = "/content/output/landing_zone_detection"

# List files in output directory
if os.path.exists(output_dir):
    print("Output directory contents:")
    for item in os.listdir(output_dir):
        item_path = os.path.join(output_dir, item)
        size = os.path.getsize(item_path) / (1024*1024)  # MB
        print(f"  {item} ({size:.2f} MB)")
else:
    print(f"Output directory not found: {output_dir}")

## 9. Test Inference

Run inference on test images to see model predictions.

In [ ]:
# Load the trained model and run inference
from rfdetr import RFDETRNano
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os

# Find the latest checkpoint
checkpoint_dir = "/content/output/landing_zone_detection"
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]

if checkpoint_files:
    latest_checkpoint = sorted(checkpoint_files)[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
    print(f"Loading checkpoint: {checkpoint_path}")
    
    # Load model
    model = RFDETRNano(
        class_names=['Landing Zone'],
        pretrain_weights=checkpoint_path
    )
    
    # Test on some images
    test_dir = "/content/coco-dataset/test"
    test_images = [f"{test_dir}/{f}" for f in os.listdir(test_dir) if f.endswith('.png')][:6]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(test_images):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Run inference
        predictions = model.predict(img_path, conf_threshold=0.3)
        
        ax = axes[idx]
        ax.imshow(img_rgb)
        
        # Draw predictions
        if predictions and len(predictions) > 0:
            for pred in predictions:
                if hasattr(pred, 'bbox'):
                    x1, y1, x2, y2 = pred.bbox
                    w, h = x2 - x1, y2 - y1
                    rect = Rectangle((x1, y1), w, h, linewidth=2, edgecolor='red', facecolor='none')
                    ax.add_patch(rect)
                    if hasattr(pred, 'confidence'):
                        ax.text(x1, y1-5, f"{pred.confidence:.2f}", color='red', fontsize=10, weight='bold')
        
        ax.set_title(os.path.basename(img_path))
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No checkpoint files found. Please train the model first.")

## 10. Save Model to Google Drive

Save the trained model back to Google Drive for later use.

In [ ]:
# Save trained model to Google Drive
import shutil
import os

output_dir = "/content/output/landing_zone_detection"
drive_output = "/content/drive/MyDrive/landing_zone_model"

if os.path.exists(output_dir):
    # Create directory in Drive
    os.makedirs(drive_output, exist_ok=True)
    
    # Copy all files
    for item in os.listdir(output_dir):
        src = os.path.join(output_dir, item)
        dst = os.path.join(drive_output, item)
        if os.path.isfile(src):
            shutil.copy2(src, dst)
            print(f"✓ Copied: {item}")
    
    print(f"\n✓ Model saved to Google Drive: {drive_output}")
    
    # Also create a zip for easy download
    shutil.make_archive("/content/drive/MyDrive/landing_zone_model_archive", 'zip', output_dir)
    print("✓ Created zip archive: /content/drive/MyDrive/landing_zone_model_archive.zip")
else:
    print("Output directory not found. Train the model first.")